In [ ]:
import pickle
import numpy as np

In [ ]:
with open("train_qa.txt","rb") as fp:
  train_data = pickle.load(fp)

In [ ]:
train_data

In [ ]:
with open("test_qa.txt","rb") as fp:
  test_data = pickle.load(fp)

In [ ]:
test_data

In [ ]:
type (test_data)

In [ ]:
type(train_data)

In [ ]:
len(test_data)

In [ ]:
len(train_data)

In [ ]:
train_data[0][2]

In [ ]:
vocab=set()

In [ ]:
all_data = test_data + train_data

In [ ]:
for story, question, answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))

In [ ]:
vocab.add('yes')
vocab.add('no')

In [ ]:
len(vocab)

In [ ]:
max_story_len = max([len(data[0])for data in all_data])
max_story_len

In [ ]:
max_ques_len = max([len(data[1])for data in all_data])
max_story_len

In [ ]:
pip install Keras-Preprocessing

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(filters=[])

In [ ]:
tokenizer.fit_on_texts(vocab)

In [ ]:
tokenizer.word_index

In [ ]:
#train dataset
train_story_text = []
train_question_text= []
train_answers = []

for story , question,answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)

In [ ]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [ ]:
train_story_seq

In [ ]:
def vectorize_stories(data, word_index = tokenizer.word_index,
                      max_story_len = max_story_len,max_ques_len = max_ques_len):

 X=[]
 Xq =[]
 Y=[]

for story, ques, ans in data:
    x= [word_index[word.lower()]for word in story]
    xq= [word_index[word.lower()]for word in ques]
    y = np.zeroes(len(word_index)+1)
    y[word_index[ans]] = 1

    X.append(x)
    Xq.append(xq)
    Y.append(y)

return (pad_sequences(X,max_len=max_story_len),
        pad_sequences(Xq,max_len=max_ques_len),
        np.array(Y))

In [ ]:
inputs_train, queries_train ,answers_train = vectorize_stories(train_data)

TypeError: ignored

In [ ]:
inputs_train

In [ ]:
inputs_test, queries_test,answers_test = vectorize_stories(test_data)

In [ ]:
vocab_len = len(vocab) + 1

In [ ]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layeyrs import Input , Activation , Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [ ]:
input_sequence = Input((max_story_len,))
question = Input((max_ques_len,))

In [ ]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len, output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [ ]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len, output_dim=max_ques_len))
input_encoder_c.add(Dropout(0.3))

In [ ]:
ques_encoder = Sequential()
ques_encoder.add(Embedding(input_dim= vocab_len, output_dim = 64,input_length=max_ques_length))
ques_encoder.add(Dropout(0.3))

In [ ]:
input_encoded_m = input_encoded(input_sequence)
input_encoded_c = input_encoder_c(question)
ques_encoded = ques_encoder(question)

In [ ]:
from keras.layers.core.activation import Activation
match = dot([input_encoded_m,question_encoded], axes =(2,2))
match = Activation ('softmax')(match)

In [ ]:
response = add([match, input_encoded_c])
response = permute((2,1))(response)

In [ ]:
answer = concatenate([response, question_encoded])

In [ ]:
from keras.layers.rnn.lstm_v1 import LSTM
answer = LSTM(32)(answer)

In [ ]:
answer=Dropout(0.5)(answer)
answer= Dense(vocab_len)(answer)

In [ ]:
answer = Activation('softmax')(answer)

In [ ]:
model = Model([input_sequence, question],answer)
model.compilr(optimizer = 'rmsprop', loss= 'categorical_crossentrophy', metrices =['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit([inputs_train,queries_train], answers_train,
                    batch_size=30, epochs = 22,
                    validation_data =([inputs_test,queries_test],answers_test)
                    )

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epochs")

In [ ]:
model.save("chatbot_model")

In [ ]:
model.load_weights("chatbot_model")

In [ ]:
pred_results= model.predict(([inputs_test, queries_test]))

In [ ]:
pred_resultls

In [ ]:
test_data[0][0]

In [ ]:
story = ' '.join(word for word in test_data[100][0])

In [ ]:
story

In [ ]:
query = ' '.join(word for word in test_data[100][1])

In [ ]:
query

In [ ]:
test_data[100][2]

In [ ]:
val_max = np.argmax(pred_results[37])

for key, val in tokenizer,word_index.items():
  if val==val_max:
    k=key

    print("Predicted answer is ",k)
    print("Probability of certainity",pred_results[37][val_max])